# 02: Create Multi-Agent System with Genie Agents

Create a supervisor-based multi-agent system using LangGraph and Databricks Genie agents.

**Pattern**: Based on [Databricks LangGraph Multi-Agent Genie Guide](https://docs.databricks.com/aws/en/generative-ai/agent-framework/multi-agent-genie)

**Key Features**:
- Supervisor agent pattern coordinating multiple Genie agents
- GenieAgent for accessing Genie Spaces (no custom UC Functions needed)
- Automatic authentication via GenieAgent
- ResponsesAgent wrapper for MLflow compatibility
- Streaming support via predict_stream()

**Architecture**:
```
Supervisor Agent
  ├─ Customer Behavior Genie Agent (space: 01f0b7572b3a185d9f69cd89bc4c7579)
  └─ Inventory Management Genie Agent (space: 01f09cdef66116e5940de4b384623be9)
```

**Prerequisites**: None! GenieAgent handles authentication automatically.

## Install Dependencies

In [ ]:
%pip install --quiet --upgrade mlflow databricks-langchain langgraph langchain-core
dbutils.library.restartPython()

## Verify Genie Space Access

**IMPORTANT**: Before creating the agent, verify you have permissions to access the Genie spaces.

This will check if you have "Can View" or "Can Use" permissions on both Genie spaces.

In [ ]:
import sys
sys.path.append('../src')

from genie.test_utils import test_all_genie_spaces
from config.agent_config import DOMAINS

# Define domain-specific test queries
test_queries = {
    "customer_behavior": "What are the top customers?",
    "inventory": "What items are at risk of overstock?"
}

# Test all Genie spaces (will wait up to 90 seconds per space)
all_passed = test_all_genie_spaces(
    domains=DOMAINS,
    test_queries=test_queries,
    max_wait_seconds=90,
    verbose=True
)

if not all_passed:
    print("\n⚠️  ACTION REQUIRED:")
    print("   1. Request 'Can Use' permission on the Genie spaces that failed")
    print("   2. Re-run this cell after permissions are granted")
    print("   3. For detailed testing, use: 10-genie-rooms/test-genie-access.ipynb")

## Create Agent Module (agent.py)

Using `%%writefile` to create standalone agent file with GenieAgent supervisor pattern.

**Note**: Only run this after verifying Genie space access above!

In [ ]:
%%writefile agent.py

import sys
sys.path.append('../src')

from typing import TypedDict, Annotated, Sequence, Any, Optional
from langchain_core.messages import AnyMessage, BaseMessage
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END
from databricks_langchain import ChatDatabricks, GenieAgent
from mlflow.pyfunc import ResponsesAgent
from mlflow.types.responses import ResponsesAgentRequest, ResponsesAgentResponse

# Import modular configuration
from config.agent_config import AGENT_CONFIG, DOMAINS
from config.prompts import SYSTEM_PROMPT


# ============================================================================
# State Management
# ============================================================================

class AgentState(TypedDict):
    """Agent state with type annotations."""
    messages: Annotated[Sequence[AnyMessage], add_messages]
    custom_inputs: Optional[dict[str, Any]]
    custom_outputs: Optional[dict[str, Any]]


# ============================================================================
# Create Genie Agents (Lazy Loading)
# ============================================================================

# Store Genie agents globally to avoid re-initialization
_genie_agents = None

def get_genie_agents():
    """Lazy-load GenieAgent instances for each domain."""
    global _genie_agents
    
    if _genie_agents is None:
        _genie_agents = {}
        
        # Customer Behavior Genie Agent
        _genie_agents["customer_behavior"] = GenieAgent(
            genie_space_id=DOMAINS["customer_behavior"].genie_space_id,
            genie_agent_name=DOMAINS["customer_behavior"].name,
            description=DOMAINS["customer_behavior"].description,
        )
        
        # Inventory Management Genie Agent
        _genie_agents["inventory"] = GenieAgent(
            genie_space_id=DOMAINS["inventory"].genie_space_id,
            genie_agent_name=DOMAINS["inventory"].name,
            description=DOMAINS["inventory"].description,
        )
    
    return _genie_agents


# ============================================================================
# Supervisor Agent
# ============================================================================

def create_supervisor_agent(llm):
    """Create supervisor agent that routes queries to Genie agents."""
    
    def supervisor_node(state: AgentState):
        """Supervisor decides which agent(s) to call."""
        # Lazy-load Genie agents here (not at import time)
        genie_agents = get_genie_agents()
        
        messages = state["messages"]
        last_message = messages[-1]
        
        # Simple keyword-based routing
        query = last_message.content.lower()
        
        responses = []
        
        # Route to customer behavior agent
        if any(keyword in query for keyword in ["cart", "abandon", "customer", "behavior", "segment", "purchase"]):
            try:
                cb_response = genie_agents["customer_behavior"].invoke({"messages": [last_message]})
                responses.append(f"[Customer Behavior]\n{cb_response['messages'][-1].content}")
            except Exception as e:
                responses.append(f"[Customer Behavior]\n[Error]: {str(e)}")
        
        # Route to inventory agent
        if any(keyword in query for keyword in ["inventory", "stock", "overstock", "stockout", "supply"]):
            try:
                inv_response = genie_agents["inventory"].invoke({"messages": [last_message]})
                responses.append(f"[Inventory]\n{inv_response['messages'][-1].content}")
            except Exception as e:
                responses.append(f"[Inventory]\n[Error]: {str(e)}")
        
        # If no specific routing, provide guidance
        if not responses:
            response_text = "I can help you with:\n- Customer behavior analysis (cart abandonment, segmentation, purchase patterns)\n- Inventory management (stock levels, stockouts, overstock)\n\nPlease ask a question related to these topics."
        else:
            response_text = "\n\n".join(responses)
        
        from langchain_core.messages import AIMessage
        return {"messages": [AIMessage(content=response_text)]}
    
    # Build simple graph with supervisor node
    workflow = StateGraph(AgentState)
    workflow.add_node("supervisor", supervisor_node)
    workflow.set_entry_point("supervisor")
    workflow.add_edge("supervisor", END)
    
    return workflow.compile()


# ============================================================================
# ResponsesAgent Wrapper
# ============================================================================

class MultiGenieResponsesAgent(ResponsesAgent):
    """ResponsesAgent wrapper for multi-Genie agent system."""
    
    def __init__(self, agent_graph):
        self.agent = agent_graph
    
    def predict(self, request: ResponsesAgentRequest) -> ResponsesAgentResponse:
        """Synchronous prediction."""
        from mlflow.types.responses import to_chat_completions_input
        
        # Convert request to LangChain messages
        messages = to_chat_completions_input([msg.model_dump() for msg in request.input])
        
        # Invoke agent
        result = self.agent.invoke({"messages": messages})
        
        # Extract response
        final_message = result["messages"][-1]
        
        return ResponsesAgentResponse(
            output=[{"type": "text", "text": final_message.content}],
            custom_outputs=request.custom_inputs
        )
    
    def predict_stream(self, request: ResponsesAgentRequest):
        """Streaming prediction."""
        from mlflow.types.responses import to_chat_completions_input, output_to_responses_items_stream
        
        # Convert request to LangChain messages
        messages = to_chat_completions_input([msg.model_dump() for msg in request.input])
        
        # Stream agent execution
        for event in self.agent.stream({"messages": messages}, stream_mode=["updates", "messages"]):
            if event[0] == "updates":
                for node_data in event[1].values():
                    if len(node_data.get("messages", [])) > 0:
                        yield from output_to_responses_items_stream(node_data["messages"])


# ============================================================================
# Agent Initialization
# ============================================================================

def initialize_agent():
    """Initialize multi-agent system with modular configuration."""
    # Initialize LLM from config
    llm = ChatDatabricks(
        endpoint=AGENT_CONFIG["model_endpoint"],
        temperature=AGENT_CONFIG["temperature"],
        max_tokens=AGENT_CONFIG["max_tokens"]
    )
    
    # Create supervisor agent (Genie agents are lazy-loaded)
    agent_graph = create_supervisor_agent(llm)
    
    # Wrap in ResponsesAgent
    return MultiGenieResponsesAgent(agent_graph)


# Create agent instance for MLflow
AGENT = initialize_agent()

In [ ]:
## Test Agent

Quick smoke test to verify the agent works before MLflow logging.

In [ ]:
from agent import AGENT
from mlflow.types.responses import ResponsesAgentRequest

# Test query - should route to inventory agent
test_query = "What products are at risk for overstock?"

print(f"Query: {test_query}\n")

# Create request with message dict
request = ResponsesAgentRequest(
    input=[{"role": "user", "content": test_query}]
)

# Test synchronous prediction
response = AGENT.predict(request)

print(f"Response:\n{response.output[0]['text']}")
print(f"\n✅ Agent test PASSED")

from mlflow.types.responses import ResponsesAgentRequest

# Test multi-domain query - should route to both agents
test_query = "What products are frequently abandoned in carts and do we have inventory issues with those items?"

# Create request
request = ResponsesAgentRequest(
    input=[{"role": "user", "content": test_query}]
)

print(f"Streaming response for: {test_query}\n")
print("Response: ")

for event in AGENT.predict_stream(request):
    if event.type == "response.output_item.done":
        print(event.item.get('text', ''), end="", flush=True)

print("\n\n✅ Streaming test PASSED")

In [ ]:
# Test multi-domain query - should route to both agents
test_query = "What products are frequently abandoned in carts and do we have inventory issues with those items?"

# Create request
request = ResponsesAgentRequest(
    input=[{"role": "user", "content": test_query}]
)

print(f"Streaming response for: {test_query}\n")
print("Response: ")

for event in AGENT.predict_stream(request):
    if event.type == "response.output_item.done":
        print(event.item.get('text', ''), end="", flush=True)

print("\n\n✅ Streaming test PASSED")

## Log Agent to MLflow

Register the agent using `mlflow.models.set_model()` (Databricks pattern).

In [ ]:
import mlflow
import sys
sys.path.append('../src')

from mlflow.models.resources import DatabricksGenieSpace
from config.agent_config import AGENT_CONFIG, DOMAINS

# Set MLflow experiment
username = spark.sql("SELECT current_user()").collect()[0][0]
experiment_name = f"/Users/{username}/multi-genie-agent"

mlflow.set_experiment(experiment_name)
print(f"MLflow experiment: {experiment_name}\n")

# Enable autolog for trace capture
mlflow.langchain.autolog()

# Prepare resources for automatic authentication passthrough
resources = []

# Add Genie spaces (GenieAgent handles authentication automatically)
for domain in DOMAINS.values():
    resources.append(DatabricksGenieSpace(genie_space_id=domain.genie_space_id))

print(f"Declaring {len(resources)} Genie Space resources for automatic auth passthrough:")
for resource in resources:
    print(f"  - {resource}")
print()

# Log agent
with mlflow.start_run(run_name="multi-genie-supervisor-agent") as run:
    # Log parameters
    mlflow.log_params({
        "model_endpoint": AGENT_CONFIG["model_endpoint"],
        "temperature": AGENT_CONFIG["temperature"],
        "max_tokens": AGENT_CONFIG["max_tokens"],
        "timeout_seconds": AGENT_CONFIG["timeout_seconds"],
        "agent_type": "multi_genie_supervisor"
    })
    
    # Log Genie space IDs as tags
    mlflow.set_tags({
        f"genie_space_{domain.name}": domain.genie_space_id
        for domain in DOMAINS.values()
    })
    
    # Import and register agent with automatic authentication passthrough
    from agent import AGENT
    mlflow.models.set_model(model=AGENT, resources=resources)
    
    run_id = run.info.run_id
    print(f"✅ Multi-agent system logged to MLflow with automatic authentication passthrough")
    print(f"Run ID: {run_id}")
    print(f"Experiment: {experiment_name}")

## Verify MLflow Registration

Check that the agent is properly registered and ready for deployment.

In [ ]:
# Get the current run
current_run = mlflow.get_run(run_id)

print("Registered Multi-Agent System Details:")
print(f"  Run ID: {current_run.info.run_id}")
print(f"  Status: {current_run.info.status}")
print(f"  Agent Type: {current_run.data.params.get('agent_type')}")
print(f"  Model Endpoint: {current_run.data.params.get('model_endpoint')}")
print(f"\n  Genie Spaces:")
for tag_key, tag_value in current_run.data.tags.items():
    if tag_key.startswith('genie_space_'):
        print(f"    - {tag_key}: {tag_value}")

print("\n✅ Multi-agent system ready for deployment")

## Next Steps

✅ Multi-agent system created with GenieAgent supervisor pattern and logged to MLflow!

**Key Improvements:**
- GenieAgent for Genie Space access (no custom UC Functions!)
- Automatic authentication via GenieAgent (no WorkspaceClient issues)
- Supervisor pattern routes queries to appropriate domain agents
- Supports multi-domain queries across both Genie spaces
- ResponsesAgent wrapper for MLflow compatibility
- Streaming support via predict_stream()

**Next**: Open `03-test-agent.ipynb` to run comprehensive tests.